In [1]:
import os
import numpy as np
import glob
import random
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import cv2 as cv
import time
import tensorflow as tf
from PIL import Image,ImageDraw

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential,Input
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D
from tensorflow.keras.losses import BinaryCrossentropy,MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras.metrics import MeanAbsoluteError, RootMeanSquaredError, MeanSquaredError
from tensorflow.keras.applications.vgg16 import VGG16

NameError: name 'custom_loss_function' is not defined

In [4]:
def custom_loss_function(y_true, y_pred):
    delta_points=tf.reduce_sum(tf.square(y_true[:2]-y_pred[:2]))
    
    h_pred=y_pred[3]-y_pred[1]
    w_pred=y_pred[2]-y_pred[0]
    h_true=y_true[3]-y_true[1]
    w_true=y_true[2]-y_true[0]
    
    delta_size=tf.reduce_sum(tf.square(h_true-h_pred)+tf.square(w_true-w_pred))
    
    return delta_points+delta_size

In [5]:
class_loss=BinaryCrossentropy()
reg_loss=custom_loss_function
opt=Adam(learning_rate=0.0001)

In [6]:
input_layer=Input(shape=(100,100,3))

vgg= VGG16(include_top=False)(input_layer)

f1=GlobalMaxPooling2D()(vgg)
class_l1=Dense(units=2048, activation='relu')(f1)
class_out= Dense(units=1,activation='sigmoid',name='class_out')(class_l1)

f2=GlobalMaxPooling2D()(vgg)
reg_l1=Dense(units=2040, activation='relu')(f2)
reg_out= Dense(units=4,activation='sigmoid',name='reg_out')(reg_l1)

model=tf.keras.Model(input_layer,[class_out,reg_out])
model.compile(loss = [class_loss,reg_loss],optimizer = opt)

In [7]:
model.load_weights('gmodel.h5')

In [ ]:
cap = cv.VideoCapture(1)
w=600
h=600

if not cap.isOpened():
    print("Cannot open camera")
    exit()

#wait for video camera to setup
time.sleep(2)

while 1:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    center = frame.shape
    x = center[1]/2 - w/2
    y = center[0]/2- h/2
    frame = frame[int(y):int(y+h), int(x):int(x+w)]
    
    img_pil = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    
    img_pil = Image.fromarray(img_pil)
    
    newsize = (100, 100)
    
    img = img_pil.resize(newsize)

    img = np.asarray(img)
    #divide by 255 and reshape to vector
    img=img/255
    
    X = np.expand_dims(img, axis=0)
    
    y_class,y_reg=model.predict(X,verbose=0)
    
    if(y_class>=0.5):
        points = (y_reg*100*6).reshape(4,)
        x_min = points[0]
        y_min = points[1]
        x_max=points[2]
        y_max=points[3]
        # w = points[2] - points[0]
        # h = points[3] - points[1]
        shape = [(x_min, y_min), (x_max, y_max)] 
        img_pil_draw=ImageDraw.Draw(img_pil)
        img_pil_draw.rectangle(shape,fill=None,outline="red")
        img_pil = img_pil_draw._image
        
        tracked = frame[int(x_min):int(x_max),int(y_min):int(y_max]

        

    frame = np.array(img_pil)
    frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
    cv.imshow("frame",frame)
    
    cv.imshow("tracked",tracked)

    
    if cv.waitKey(1) == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()
# I don't know why we need these 4 lines but it works
cv.waitKey(1)
cv.waitKey(1)
cv.waitKey(1)
cv.waitKey(1)

2023-12-02 07:56:44.539 Python[5089:297214] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


## 